In [2]:
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.windows import from_bounds
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import numpy as np
from patchify import patchify
import torch
from torch.functional import F
from torch import nn
from torch.utils.data import DataLoader
from shapely.geometry import box
from torchvision import transforms
import time
import random

from pytorch_segmentation.data.rwanda_dataset import RwandaDataset
from pytorch_segmentation.data.inmemory_dataset import InMemorySatDataset
from pytorch_segmentation.train_net import train
from pytorch_segmentation.models import UNet
import pytorch_segmentation.augmentation.transforms as seg_transforms

seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [2]:

data_path = "/home/jovyan/work/satellite_data/tmp/2018.vrt"
label_path = "data/label_masks/train"
test_label_path = "data/label_masks/test"
tensorboard_path = "/home/jovyan/work/runs"

test_patch_size = [256,256,3]
overlap = 192
padding = False#True


#batch_size = 200
#batch_size = 50
batch_size = 150

nworkers = 4
pin_memory = True

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

save_dir = "saved_models"


test_transform = None
#test_transform = seg_transforms.Normalize(mean=[0.5492, 0.5190, 0.4393],
#                                          std=[0.1381, 0.1320, 0.1349])

model_name = "unet_05_05_2022_113034"
model_path = save_dir+"/"+model_name+".pth"

# 1.1 Data Collection

In [4]:
test_dataset = InMemorySatDataset(data_file_path=data_path,mask_path=label_path,
                             overlap=overlap,patch_size=test_patch_size,padding=padding,transform=None)

# 2 Model

In [5]:
# Change here to adapt to your data
# n_channels=3 for RGB images 
# n_classes is the number of probabilities you want to get per pixel


net = UNet(n_channels=3, n_classes=2, bilinear=False)
#net= DataParallel(net)

net.load_state_dict(torch.load(model_path))

net = net.to(device=device)



## 3 Validation

In [6]:
test_dl = DataLoader(test_dataset,batch_size=batch_size,num_workers=nworkers,pin_memory=pin_memory,drop_last=False)

In [7]:
net.eval()
output = []
y_test = []

with torch.no_grad():
    for i,batch in enumerate(test_dl):
        x = batch["x"].to(device)
        out = net(x)#["out"]
        #out = F.softmax(out,dim=1)
        out = torch.argmax(out,dim=1)
        out = out.cpu().numpy()
        output.append(out)
        y_test.extend(batch["y"].numpy())
    y_pred = torch.as_tensor(np.vstack(output))
    y_test = torch.as_tensor(np.array(y_test))

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [9]:
def iou_score(y_pred,target,absent_score=1.0):
    numerator = torch.sum(y_pred * target,dim=1)  # TP
    denominator = torch.sum(y_pred + target,dim=1) - numerator  # 2TP + FP + FN - TP
    iou = (numerator) / (denominator)
    iou[denominator == 0] = absent_score
    return iou.mean()

def dice_coeff(y_pred, target,absent_score=1.0):
    # F1 = TP / (TP + 0.5 (FP + FN)) = 2TP / (2TP + FP + FN)
    numerator = 2 * torch.sum(y_pred * target,dim=1)  # 2TP
    denominator = torch.sum(y_pred + target,dim=1)  # 2TP + FP + FN
    dice = (numerator) / (denominator)
    dice[denominator == 0] = absent_score
    return dice.mean()


In [10]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc},**test_dataset.get_config()}])
df.to_csv(model_path+"_score.csv")

Model:  unet_05_05_2022_113034
IoU:  0.7121449112892151
Dice score:  0.7348865866661072
Accuracy:  0.9825443625450134


# Models

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc},**test_dataset.get_config()}])
df.to_csv(model_path+"_score.csv")

Model:  unet_09_05_2022_110252
IoU:  0.835816502571106
Dice score:  0.8585439324378967
Accuracy:  0.9832549691200256


In [10]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc},**test_dataset.get_config()}])
df.to_csv(model_path+"_score.csv")

Model:  unet_05_05_2022_113034
IoU:  0.7121449112892151
Dice score:  0.7348865866661072
Accuracy:  0.9825443625450134


In [ ]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc},**test_dataset.get_config()}])
df.to_csv(model_path+"_score.csv")

Model:  unet_06_05_2022_070627
IoU:  0.8286783695220947
Dice score:  0.851495623588562
Accuracy:  0.9829235672950745


In [10]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc},**test_dataset.get_config()}])
df.to_csv(model_path+"_score.csv")

Model:  unet_06_05_2022_071606
IoU:  0.8318414688110352
Dice score:  0.8548348546028137
Accuracy:  0.983232319355011


In [10]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc},**test_dataset.get_config()}])
df.to_csv(model_path+"_score.csv")

Model:  unet_06_05_2022_070624
IoU:  0.8266518115997314
Dice score:  0.849705159664154
Accuracy:  0.9835194945335388


In [11]:
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score

y_pred = y_pred.reshape(y_pred.size(0),-1)
y_test = y_test.reshape(y_test.size(0),-1)

print("Model: ",model_name)
iou = iou_score(y_pred,y_test).item()
print(f"IoU: ",iou)
dice = dice_coeff(y_pred,y_test).item()
print("Dice score: ",dice)
acc = (torch.sum(y_pred == y_test)/torch.numel(y_pred)).item()
print("Accuracy: ",acc)
# f1 = f1_score(y_pred.reshape(-1).numpy(),y_test.reshape(-1).numpy())
# print("F1 score: ",f1)
#accuracy = accuracy(imgs,test_dataset.y)




#print("Pixel accuracy: ",dice_score)
df = pd.DataFrame([{**{"model_name":model_path,"IoU":iou,"dice_score":dice,
                       "Accuracy":acc},**test_dataset.get_config()}])
df.to_csv(model_path+"_score.csv")

Model:  unet_06_05_2022_072927
IoU:  0.8136389255523682
Dice score:  0.8368015289306641
Accuracy:  0.9829322099685669
